# Reading and editing a topography
A topograph/bathymetry is needed to create the domain of interpolation (land-sea mask).     
The topography/bathymetry is usually extracted from a netCDF file.     
Adapted topographies:
* General Bathymetric Chart of the Oceans ([GEBCO](https://www.gebco.net/))
* [EMODnet Bathymetry](http://www.emodnet-bathymetry.eu)
* Other topographies can be used as long as they define the netCDF variables `lon`, `lat` and `bat` (positive above sea-level and negative below, in `DIVAnd` however, the convention is that `z` is positive in sea water).

This notebook illustrates how the land-sea mask can be modified, for example to exclude a lake.

In [ ]:
using DIVAnd
using PyPlot
using Dates
using Statistics

## Data download and domain selection 
The links for the different versions of the bathymetry are:      
https://dox.uliege.be/index.php/s/RSwm4HPHImdZoQP/download → gebco_30sec_4.nc      
https://dox.uliege.be/index.php/s/wS6Y8P8NhIF60eG/download → gebco_30sec_8.nc      
https://dox.uliege.be/index.php/s/U0pqyXhcQrXjEUX/download → gebco_30sec_16.nc        

In [ ]:
bathname = "../data/gebco_30sec_16.nc"
if !isfile(bathname)
    @info("Download bathymetry")
    download("https://dox.uliege.be/index.php/s/U0pqyXhcQrXjEUX/download", bathname)
else
    @info("Bathymetry file already downloaded")
end
bathisglobal = true;

We have two main tools to load the bathymetry:
1. `DIVAnd.extract_bath` loads the bathymetry at the resolution defined in the netCDF file while 
2. `DIVAnd.load_bath` reduces the resolution to match the resolution of the analysis.

### Set domain

In [ ]:
dx = dy = 0.125
lonr = 0.0:dx:10.0
latr = 52.3:dy:55.6

### Extract bathymetry

In [ ]:
bx,by,b = extract_bath(bathname,bathisglobal,lonr,latr)
@show size(b);

### Make a plot

In [ ]:
pcolor(bx, by, permutedims(b, [2, 1]), vmin=0.0); 
colorbar()
ax = gca()
ax.set_aspect(1/cos(mean([ylim()...]) * pi/180))

### 🖋️ Modify the domain

In [ ]:
dx = ...
dy = ...
lonr = ...
latr = ...

### 🖋️ Re-run the extraction 

In [ ]:
bx,by,b = ...

### 🖋️ Plot the results

In [ ]:
pcolor(...)

### Load bathymetry
Same as `extract_bath`, but change its original resolution and match it to that of the analysis, defined by `lonr` and `latr`.

In [ ]:
bx,by,b = load_bath(bathname,true,lonr,latr)
@show size(b)

pcolor(bx,by,permutedims(b, [2, 1])); 
colorbar(orientation="horizontal")
gca().set_aspect(1/cos(mean([ylim()...]) * pi/180))

## Mask creation
We first set a list of vertical levels.

In [ ]:
depthr = [0.0, 10., 20., 500.]

Then we use the `load_mask` function to get the 3D mask.

In [ ]:
xmask, ymask, mask = load_mask(bathname, true, lonr, latr, depthr);

### 🖋️ Plot the mask at different depth levels

In [ ]:
# 1st level:
pcolor(xmask, ymask, Float64.(permutedims(mask[:,:,1], [2, 1])), cmap=PyPlot.cm.binary)
gca().set_aspect(1/cos(mean([ylim()...]) * pi/180)) 

In [ ]:
# Another level...
pcolor(...)
gca().set_aspect(1/cos(mean([ylim()...]) * pi/180)) 

## Mask editing (if we have time)
To edit the mask, you can either modify directly its value if you know the indices, or you can use a selection based on the coordinates.
### Create an artificial island

In [ ]:
sellon = findall((lonr .<= 4.5) .& (lonr .>= 3.5));
sellat = findall((latr .<= 55.2) .& (latr .>= 54.3));
mask2 = copy(mask);
mask2[sellon, sellat,:] .= false;

### 🖋️ Make the plot for the new mask

In [ ]:
...